In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from bs4 import BeautifulSoup
import gzip
from pathlib import Path
from datetime import datetime
import re
from glob import glob
import pandas as pd
from time import sleep
from random import uniform

In [2]:
url = 'https://dunksandthrees.com/'
chromedriver_path = Path().joinpath(
    '/Users','johnschroeder','Documents','gambling_stuff','chromedriver')
base_path = '/Users/johnschroeder/Documents/gambling_stuff/data/01_raw/nba_team_ratings/dunks_and_threes'

In [3]:
chrome_options = ChromeOptions()
# chrome_options.add_argument('--headless')

In [4]:
driver = Chrome(executable_path=chromedriver_path, options=chrome_options)

In [5]:
response = driver.get(url)

In [6]:
soup = BeautifulSoup(driver.page_source)
raw_page_date = soup.find('span', {'class': 'page-date'}).text
print(raw_page_date)
ymd = str(datetime.strptime(raw_page_date, '%B %d, %Y').date()).split('-')
raw_file = Path(base_path).joinpath(*ymd, 'dunksandthrees.html')
team_stats = soup.find('div', {'id': 'team-stats'})
Path(raw_file).parent.mkdir(parents=True, exist_ok=True)
with open(raw_file, 'w') as fp:
    fp.write(str(team_stats.prettify()))

April 15, 2020


In [9]:
# iterate through multiple clicks to pull multiple raw files
nbr_clicks = 12

for click in range(nbr_clicks):
    soup = BeautifulSoup(driver.page_source)
    raw_page_date = soup.find('span', {'class': 'page-date'}).text
    print(raw_page_date)
    ymd = str(datetime.strptime(raw_page_date, '%B %d, %Y').date()).split('-')
    raw_file = Path(base_path).joinpath(*ymd).with_suffix('.html')
    team_stats = soup.find('div', {'id': 'team-stats'})
    Path(raw_file).parent.mkdir(parents=True, exist_ok=True)
    with open(raw_file, 'w') as fp:
        fp.write(str(team_stats.prettify()))
    driver.find_element_by_class_name('date-button').click()
    sleep(uniform(1.5, 2.5))

November  2, 2019
November  1, 2019
October 31, 2019
October 30, 2019
October 29, 2019
October 28, 2019
October 27, 2019
October 26, 2019
October 25, 2019
October 24, 2019
October 23, 2019
October 22, 2019


In [115]:
# collect all raw files and combine to a single dataframe

In [10]:
curated_file = '/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv'

In [11]:
stats_headers = team_stats.find('tr', {'class': 'main-header-tr'})

In [12]:
df_headers = [re.sub('\W+', '', _.text).lower() for _ in stats_headers.find_all('th')]

In [13]:
df_headers

['team',
 'wl',
 'anet',
 'aortg',
 'adrtg',
 'sos',
 'osos',
 'dsos',
 'pace',
 'opl',
 'dpl']

In [14]:
df_headers1 = ['team',
 'wl',
 'anet',
 'aortg',
 'adrtg',
 'sos',
 'osos',
 'dsos',
 'pace',
 'opl',
 'dpl',
 'date']

In [15]:
raw_files = glob('/Users/johnschroeder/Documents/gambling_stuff/data/01_raw/nba_team_ratings/dunks_and_threes/*/*/*.html')
df_data = []
for raw_file in raw_files:
    with open(raw_file, 'r') as fp:
        raw_html = fp.read()
    team_stats = BeautifulSoup(raw_html)
    team_rows = team_stats.find_all('tr', {'class': 'stat-row'})
    replaces = re.compile(r'(\+)|(\s+)')
    file_date = '-'.join(Path(raw_file).with_suffix('').parts[-3:])
    for team_row in team_rows:
        row_data = [re.sub(replaces, '', _.text).strip() for _ in team_row.find_all('td')]
        row_data.append(file_date)
        df_data.append(row_data)

In [16]:
incr_df = pd.DataFrame(df_data, columns=df_headers1)

In [17]:
incr_df['team'].unique()

array(['1HOU', '2GSW', '4OKC', '2BOS', '1TOR', '3POR', '8WAS', '11NYK',
       '5IND', '9DET', '7SAS', '5UTA', '9DEN', '3PHI', '14ORL', '8MIN',
       '10CHA', '6NOP', '10LAC', '4CLE', '14MEM', '11LAL', '7MIL',
       '12BKN', '6MIA', '15ATL', '13DAL', '15PHX', '13CHI', '12SAC',
       '1MIL', '1GSW', '2DEN', '2TOR', '4BOS', '6OKC', '4HOU', '8LAC',
       '13NOP', '8DET', '10MIA', '11MIN', '9SAC', '7ORL', '6BKN', '14DAL',
       '9CHA', '10LAL', '12MEM', '11WAS', '12ATL', '15NYK', '14CLE',
       '1LAL', '7DAL', '3BOS', '5MIA', '2LAC', '10PHX', '3DEN', '6UTA',
       '4IND', '6PHI', '5OKC', '10WAS', '8ORL', '14MIN', '7BKN', '8POR',
       '11CHI', '11SAS', '13DET', '12NYK', '15CLE', '9MEM', '14ATL',
       '15GSW', '1UTA', '3MIL', '2PHX', '3LAC', '4DEN', '1PHI', '7LAL',
       '2BKN', '5ATL', '5DAL', '7BOS', '11NOP', '4NYK', '10IND', '10SAS',
       '12TOR', '8GSW', '6POR', '8CHA', '15DET', '9WAS', '13OKC', '15HOU',
       '13ORL', '2PHI', '1BKN', '5LAL', '6DAL', '6BOS', '8MEM', '7POR'

In [18]:
incr_df['team'] = incr_df['team'].replace(regex='\d+', value='')

In [21]:
incr_df.head()

,team,wl,anet,aortg,adrtg,sos,osos,dsos,pace,opl,dpl,date
0,HOU,14-4,10.21,114.51,104.39,0.38,0.16,0.29,99.08,14.18,15.027,2017-11-23
1,GSW,13-5,9.52,113.82,104.38,0.44,0.23,0.28,102.64,13.72,14.48,2017-11-23
2,OKC,8-9,5.73,106.715,101.03,0.111,-0.222,0.37,96.920,14.310,15.430,2017-11-23
3,BOS,16-3,5.64,105.618,100.01,-1.829,-0.120,-1.730,96.524,15.021,14.822,2017-11-23
4,TOR,11-6,5.55,111.54,106.013,-0.526,-0.227,-0.324,98.215,14.716,14.718,2017-11-23


In [22]:
incr_df.sort_values('date', ascending=False).to_csv('/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv', index=False)